In [1]:
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI

import os

with open('/home/gui/Área de Trabalho/DS_challenge/DS_challenge/SourceFiles/key', 'r') as f:
    os.environ['GOOGLE_API_KEY'] = f.read()

In [2]:
df = pd.read_excel('/home/gui/Área de Trabalho/DS_challenge/DS_challenge/SourceFiles/preguntas_mercadolibre.xlsx')
df.head()

,SITE,PREGUNTA,PUBLICACION
0,MLC,"Hola! La mascarilla es de 500 ml, es la origin...",title: Mascarilla Para El Cabello Karseell Col...
1,MLM,"Hola, es nuevo, facturan, tiene garantía?",title: Cámara Termográfica Flir Tg297 Point 19...
2,MLC,"Hola, vienen con caja? Son de buena calidad? T...",title: Zapatos Amiu Nuevos De Verano 2024 Para...
3,MLB,Vocês emitem nota fiscal da compra ?,title: Rodizio Borracha Roda Maciça 250-4 Rol....
4,MLM,Hice compra y quisiera factura,title: Manguera De Jardín Expandible De Hasta ...


In [3]:
prompt_template = """Você é um assistente virtual de e-commerce. Sua tarefa é responder perguntas de clientes sobre um produto, utilizando estritamente as informações fornecidas.

[INFORMAÇÕES DA PUBLICAÇÃO DO PRODUTO]
{context}

[PERGUNTA DO CLIENTE]
{question}

Instruções:
- Use apenas o contexto fornecido para responder.
- Se a pergunta não puder ser respondida com as informações fornecidas, responda educadamente "Não tenho essa informação disponível para este produto."
- Mantenha a resposta clara, concisa e no mesmo idioma da pergunta.
"""

In [4]:
try:
    llm_model = ChatGoogleGenerativeAI(
                    model="gemini-2.5-flash",
                    temperature=0,
                    max_tokens=None,
                    timeout=None,
                    max_retries=2,
                )
        
        # Criando uma função para interagir com o modelo, aplicando o prompt
    def get_gemini_response(prompt_text):
        return llm_model.generate_content(prompt_text).text
except Exception as e:
    print(f"Erro ao carregar o LLM do Gemini: {e}")
    llm_model = None


In [5]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
qa_chain = LLMChain(llm=llm_model, prompt=PROMPT)

df['resposta'] = df.apply(lambda x: qa_chain.invoke({'question': x.PREGUNTA, 'context': x.PUBLICACION})['text'], axis=1)
df.head()

/tmp/ipykernel_17396/2866166003.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  qa_chain = LLMChain(llm=llm_model, prompt=PROMPT)


,SITE,PREGUNTA,PUBLICACION,resposta
0,MLC,"Hola! La mascarilla es de 500 ml, es la origin...",title: Mascarilla Para El Cabello Karseell Col...,"Olá!\n\nSim, a mascarilla é de 500ml, conforme..."
1,MLM,"Hola, es nuevo, facturan, tiene garantía?",title: Cámara Termográfica Flir Tg297 Point 19...,"Hola. Sí, el artículo es nuevo, facturamos y c..."
2,MLC,"Hola, vienen con caja? Son de buena calidad? T...",title: Zapatos Amiu Nuevos De Verano 2024 Para...,Olá!\n\nNão tenho essa informação disponível p...
3,MLB,Vocês emitem nota fiscal da compra ?,title: Rodizio Borracha Roda Maciça 250-4 Rol....,Não tenho essa informação disponível para este...
4,MLM,Hice compra y quisiera factura,title: Manguera De Jardín Expandible De Hasta ...,"Sim, emitimos factura electrónica para todos n..."
